In [ ]:
# !pip install kor==0.8.0 markdownify
!pip uninstall kor==0.8.0 markdownify -y


In [ ]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



In [ ]:
from duckduckgo_search import ddg

keywords = 'Wer ist der Direktor des ITZBund?'
results = ddg(keywords, region='de-de', safesearch='On', time='y', max_results=3)
results

In [ ]:
from langchain.utilities import TextRequestsWrapper
from kor.documents.html import MarkdownifyHTMLProcessor
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


# requests = TextRequestsWrapper()
# html = requests.get(results[0]['href'])
html = urllib.request.urlopen(results[0]['href']).read()
res = text_from_html(html)
doc = Document(page_content=html)

md = MarkdownifyHTMLProcessor(("svg", "img", "script", "style","a")).process(doc)

doc = Document(page_content=md.page_content.replace("\n", " ").replace("  ", " ").replace("|", "").replace("*", "").replace("-",""))
split_docs = RecursiveCharacterTextSplitter().split_documents([doc])


print(doc.page_content)